In [2]:
from agent import QLearningAgent
from verify_data import FishGame, ParseError

In [ ]:
import os
import pickle
import torch
import multiprocessing as mp

def process_file(filename, data_dir='data'):
    filepath = os.path.join(data_dir, filename)
    file_memories = []
    try:
        with open(filepath, 'r') as f:
            print(f"{filename}")
            game = FishGame(f.readlines())
            for player in game.players:
                for _ in range(100):
                    game.shuffle()
                    file_memories.append(game.memory(player))
        return file_memories
    except ParseError as e:
        print(f"{filename}: {e}")
        return []

if os.path.isfile('memories_extended.pkl'):
    with open('memories_extended.pkl', 'rb') as f:
        memories = pickle.load(f)
else:
    memories = []
    filenames = os.listdir('data')
    
    if torch.cuda.is_available():
        num_processes = min(mp.cpu_count(), 8)
        with mp.Pool(processes=num_processes) as pool:
            results = pool.map(process_file, filenames)
        for result in results:
            memories.extend(result)
    else:
        for filename in filenames:
            filepath = os.path.join('data', filename)
            with open(filepath, 'r') as f:
                try:
                    print(f"{filename}")
                    game = FishGame(f.readlines())
                    for player in game.players:
                        for _ in range(100):
                            game.shuffle()
                            memories.append(game.memory(player))
                except ParseError as e:
                    print(f"{filename}: {e}")
                    break
    with open('memories_extended.pkl', 'wb') as f:
        pickle.dump(memories, f)

12-3_11:12.txt
1-15_11:15.txt
12-3_11:30.txt
12-6_11:08.txt
12-4_11:11.txt
12-3_14:27.txt
12-10_11:07.txt
12-3_15:27.txt
12-3_14:05.txt


In [ ]:
import pickle
with open('call_memories.pkl', 'rb') as f:
    call_memories = pickle.load(f)
with open('stored_memories.pkl', 'rb') as f:
    stored_memories = pickle.load(f)

In [2]:
from agent import QLearningAgent
import pickle
with open('test_memories.pkl', 'rb') as f:
    call_memories = pickle.load(f)
agent = QLearningAgent()
agent.train_on_data(call_memories[::100], 10, 0)

Memory loaded in 0.0 seconds


Training Q-Network epoch 9 train loss 0.24511 test loss 0.33334 lr 0.0008: 100%|██████████| 10/10 [00:01<00:00,  6.46it/s]


In [ ]:
agent = QLearningAgent()
agent.load_model('models/fish_agent.pth')

In [ ]:
agent.train_on_data(stored_memories[::10], 1, 0)

In [ ]:
from datetime import datetime

os.makedirs('models', exist_ok=True)
model_path = f'models/fish_agent_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pth'
agent.save_model(model_path)